In [30]:
import os
import requests
from yt_dlp import YoutubeDL

def download_videos_one_by_one(channel_url, save_path="./", playlist_items='1-2', cookies_path="cookies.txt", max_duration=300):
    try:
        # Step 1: 提取频道信息
        ydl_opts_info = {
            'quiet': False,  # 显示调试信息
            'extract_flat': True,  # 只提取视频信息，不下载视频
            'playlist_items': playlist_items,  # 选择指定范围的视频
        }
        with YoutubeDL(ydl_opts_info) as ydl:
            info_dict = ydl.extract_info(channel_url, download=False)
            videos = info_dict.get('entries', [])
            
            if not videos:
                print("No videos found.")
                return

        # Step 2: 逐个处理符合条件的视频
        for video in videos:
            video_url = video.get('url')
            video_id = video.get('id')
            video_title = video.get('title')
            video_duration = video.get('duration', 0)

            # 检查视频时长
            if video_duration > max_duration:
                print(f"Skipping video (too long): {video_title}")
                continue

            # 创建视频保存路径
            video_folder = save_path

            print(f"Downloading video: {video_title} ({video_url})")

            # 下载视频
            ydl_opts_video = {
                'outtmpl': os.path.join(video_folder, f"%(title)s.%(ext)s"),  # 视频保存路径
                'format': 'bestvideo+bestaudio/best',                       # 最佳视频+最佳音频
                'cookiesfrombrowser': ('chrome',),                         # 从浏览器加载 cookies
            }

            try:
                with YoutubeDL(ydl_opts_video) as ydl:
                    ydl.download([video_url])
                print(f"Download completed for: {video_title}")
            except Exception as e:
                print(f"Failed to download video: {video_title} ({e})")
                continue

            # 获取高质量缩略图
            thumbnail_url = None
            thumbnails = video.get('thumbnails', [])
            
            # 优先尝试 maxresdefault.jpg
            if video_id:
                maxres_url = f"https://img.youtube.com/vi/{video_id}/maxresdefault.jpg"
                response = requests.head(maxres_url)
                if response.status_code == 200:
                    thumbnail_url = maxres_url
                else:
                    print(f"Max resolution thumbnail not available for: {video_title}")

            # 回退到提供的最高分辨率缩略图
            if not thumbnail_url and thumbnails:
                highest_res = max(thumbnails, key=lambda t: t.get('height', 0) * t.get('width', 0))
                thumbnail_url = highest_res['url']

            if thumbnail_url:
                try:
                    thumbnail_path = os.path.join(video_folder, f"{video_title}.jpg")
                    response = requests.get(thumbnail_url, stream=True)
                    if response.status_code == 200:
                        with open(thumbnail_path, 'wb') as f:
                            for chunk in response.iter_content(1024):
                                f.write(chunk)
                        print(f"Thumbnail downloaded for: {video_title}")
                    else:
                        print(f"Failed to download thumbnail for: {video_title}")
                except Exception as e:
                    print(f"Error downloading thumbnail for: {video_title} ({e})")
            else:
                print(f"No thumbnail found for: {video_title}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
channel_url = "https://www.youtube.com/@NBA/videos"
save_path = "./downloads"
playlist_items = '1-2'  # 下载前 5 个视频及其封面
download_videos_one_by_one(channel_url, save_path, playlist_items)
